In [1]:
# Import necessary libraries
import pandas as pd
import requests
from datetime import datetime, timedelta
from time import sleep
from tqdm import tqdm
import psycopg2

# Load the dataset
file_path = "ethereum_txs_lite.csv"
data = pd.read_csv(file_path)


# 1. Compute the approximate execution timestamp of each transaction
block_length = 12  # seconds
# Convert block_timestamp to datetime object for arithmetic operations
data['block_timestamp'] = pd.to_datetime(data['block_timestamp'])
# Compute execution timestamp
data['execution_timestamp'] = data['block_timestamp'] + \
    (data['transaction_index'] * timedelta(seconds=block_length/len(data)))

# 2. Compute the gas cost of each transaction in Gwei
data['gas_cost'] = data['gas'] * data['gas_price'] * 1e-9  # Gwei to ETH

# 3. Retrieve the approximate price of ETH at transaction execution time and compute the dollar cost of gas used
# Define a function to get ETH price from Coingecko
def get_eth_price(timestamp):
    url = f"https://api.coingecko.com/api/v3/simple/price?ids=ethereum&vs_currencies=usd&at={timestamp}"
    retry_count = 0
    max_retries = 5
    backoff_factor = 2  # Exponential backoff factor
    sleep_time = 5  # initial sleep time of 1 second
    
    while retry_count < max_retries:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()['ethereum']['usd']
        elif response.status_code == 429:
            sleep(sleep_time)
            sleep_time *= backoff_factor  # Exponential backoff
            sleep_time = min(sleep_time, 60)  # Cap the sleep time at 60 seconds
            retry_count += 1
        else:
            print(f"Failed to retrieve data: {response.status_code}")
            return None
    print(f"Failed to retrieve data after {max_retries} retries")
    return None

# Get unique timestamps to minimize API calls
unique_timestamps = data['execution_timestamp'].dt.strftime('%s').unique()
eth_prices = {}

for ts in tqdm(unique_timestamps):
    eth_prices[ts] = get_eth_price(ts)

# Map the ETH prices back to the DataFrame
data['eth_price_usd'] = data['execution_timestamp'].dt.strftime('%s').map(eth_prices)
# Compute the dollar cost of gas used
data['gas_cost_usd'] = data['gas_cost'] * data['eth_price_usd']

# Display the first few rows of the DataFrame to verify the calculations
data.head()

100%|██████████| 12/12 [00:00<00:00, 15.54it/s]


,hash,nonce,block_hash,block_number,transaction_index,from_address,to_address,value,gas,gas_price,...,receipts_cumulative_gas_used,receipts_gas_used,receipts_contract_address,receipts_root,receipts_status,receipts_effective_gas_price,execution_timestamp,gas_cost,eth_price_usd,gas_cost_usd
0,0xc055b65e39c15e1bc90cb4ccb2daac6b59c02ec1aa6c...,891,0x73c053dc4b54ece7ef678a6488a56b5772c0f2e84ce2...,17818542,0,0xd5e87f1f003f222188cc8c5aeefc8b285738b7e7,0xf24a5cc235e5242d69fafbffd304f63b92ac82f9,0,1000000,23759870228,...,295582,295582,NaN,NaN,1,23759870228,2023-08-01 07:04:59+00:00,2.375987e+07,1576.85,3.746575e+10
1,0x6f218a5e009c56f8db17e933af7cc98360b699ae88cb...,7,0x73c053dc4b54ece7ef678a6488a56b5772c0f2e84ce2...,17818542,1,0x62caee1f532bea1135733a909f1cbe4e0abf282b,0x3999d2c5207c06bbc5cf8a6bea52966cabb76d41,10000000000000000,600000,25252522390,...,460047,164465,NaN,NaN,1,25252522390,2023-08-01 07:04:59.631579+00:00,1.515151e+07,1576.85,2.389166e+10
2,0x5aa235c2497c45bf7ebce7429e837c46bf5f1716b065...,187506,0x73c053dc4b54ece7ef678a6488a56b5772c0f2e84ce2...,17818542,2,0x1c8f6a5f009e051cab9c3851ca2da2c936b2775a,0xa69babef1ca67a37ffaf7a485dfff3382056e78c,41984,219786,19252522390,...,569917,109870,NaN,NaN,1,19252522390,2023-08-01 07:05:00.263158+00:00,4.231435e+06,1570.99,6.647542e+09
3,0x5e34af06faedad87aded467547fd3f4f1d6f76d58e95...,159493,0x73c053dc4b54ece7ef678a6488a56b5772c0f2e84ce2...,17818542,3,0x43e4715ae093a4c86b5ecddb52216c4f879e9672,0xa69babef1ca67a37ffaf7a485dfff3382056e78c,28928,206662,19252522390,...,673240,103323,NaN,NaN,1,19252522390,2023-08-01 07:05:00.894737+00:00,3.978765e+06,1570.99,6.250600e+09
4,0x99b519673b88aa88e432caacdac0a9ac3630aaa2028e...,45390,0x73c053dc4b54ece7ef678a6488a56b5772c0f2e84ce2...,17818542,4,0x0e747eb2ff0f26fb77c3a1ea67ee07fac2dbb783,0xdac17f958d2ee523a2206206994597c13d831ec7,0,101134,28541372994,...,736449,63209,NaN,NaN,1,28541372994,2023-08-01 07:05:01.526316+00:00,2.886503e+06,1570.99,4.534668e+09


## Dump to db

In [2]:
# Database connection
conn = psycopg2.connect(
    dbname='transactions_db',
    user='user',
    password='password',
    host='postgres'
)

def create_tx_table():
    with conn.cursor() as cursor:
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS transactions (
                id SERIAL PRIMARY KEY,
                hash VARCHAR(255),
                from_address VARCHAR(255),
                to_address VARCHAR(255),
                block_number INT,
                executed_at TIMESTAMP,
                gas_used BIGINT,
                gas_cost_in_dollars DOUBLE PRECISION
            );
        """)
    conn.commit()

create_tx_table()  # Create the transactions table

In [3]:
def insert_transaction(row):
    with conn.cursor() as cursor:
        cursor.execute("""
            INSERT INTO transactions (
                hash, from_address, to_address, block_number,
                executed_at, gas_used, gas_cost_in_dollars
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (
            row['hash'],
            row['from_address'],
            row['to_address'],
            row['block_number'],
            row['execution_timestamp'],
            row['gas'],
            row['gas_cost_usd']
        ))
    conn.commit()

# Iterate over DataFrame rows and insert transactions into the database
for idx, row in tqdm(data.iterrows(), total=len(data)):
    insert_transaction(row)

100%|██████████| 19/19 [00:00<00:00, 1254.18it/s]


In [4]:
## Pull the txs from the DB to verify

def fetch_all_transactions(): # This is not ideal if you have too many of them
    with conn.cursor() as cursor:
        cursor.execute("SELECT * FROM transactions;")
        columns = [desc[0] for desc in cursor.description]  # Get column names from the cursor description
        data = cursor.fetchall()
    return pd.DataFrame(data, columns=columns)

# Fetch all transactions from the database
fetched_data = fetch_all_transactions()

# Display the first few rows of the fetched data
fetched_data.head()

,id,hash,from_address,to_address,block_number,executed_at,gas_used,gas_cost_in_dollars
0,1,0xc055b65e39c15e1bc90cb4ccb2daac6b59c02ec1aa6c...,0xd5e87f1f003f222188cc8c5aeefc8b285738b7e7,0xf24a5cc235e5242d69fafbffd304f63b92ac82f9,17818542,2023-08-01 07:04:59.000000,1000000,3.782809e+10
1,2,0x6f218a5e009c56f8db17e933af7cc98360b699ae88cb...,0x62caee1f532bea1135733a909f1cbe4e0abf282b,0x3999d2c5207c06bbc5cf8a6bea52966cabb76d41,17818542,2023-08-01 07:04:59.002397,600000,2.412272e+10
2,3,0x5aa235c2497c45bf7ebce7429e837c46bf5f1716b065...,0x1c8f6a5f009e051cab9c3851ca2da2c936b2775a,0xa69babef1ca67a37ffaf7a485dfff3382056e78c,17818542,2023-08-01 07:04:59.004794,219786,6.736867e+09
3,4,0x5e34af06faedad87aded467547fd3f4f1d6f76d58e95...,0x43e4715ae093a4c86b5ecddb52216c4f879e9672,0xa69babef1ca67a37ffaf7a485dfff3382056e78c,17818542,2023-08-01 07:04:59.007191,206662,6.334591e+09
4,5,0x99b519673b88aa88e432caacdac0a9ac3630aaa2028e...,0x0e747eb2ff0f26fb77c3a1ea67ee07fac2dbb783,0xdac17f958d2ee523a2206206994597c13d831ec7,17818542,2023-08-01 07:04:59.009588,101134,4.595602e+09


In [5]:
fetched_data["hash"].values[0]

'0xc055b65e39c15e1bc90cb4ccb2daac6b59c02ec1aa6c4216276054b4f31ed90a'